<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/4%20Aprendizaje%20reforzado/Sesion%202/Expected_sarsa_alm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Espected Sarsa


Este cuaderno se basa en `qlearning.ipynb` para implementar el Valor esperado SARSA.

La política que usaremos es la política de épsilon-greedy, donde el agente toma medidas óptimas con probabilidad $ (1-\epsilon)$, de lo contrario, muestra la acción al azar. Ten en cuenta que el agente __puede ocasionalmente muestrear una acción óptima durante el muestreo aleatorio por pura casualidad.

In [ ]:
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1
        
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

bash: ../xvfb: No such file or directory
env: DISPLAY=:1


In [ ]:
class QLearningAgent:
    def __init__(self, alpha, epsilon, discount, get_legal_actions):

        """
        Agente de Q-Learning
        Variables de instancia a las que tiene acceso
          - self.epsilon (problema de exploración)
          - self.alpha (tasa de aprendizaje)
          - self.discount (tasa de descuento aka gamma)

        Funciones que debes usar
          - self.get_legal_actions (state) {estado, hashable -> lista de acciones, cada una es hashable}
            que devuelve acciones legales para un estado
          - self.get_qvalue (estado, acción)
            que devuelve Q (estado, acción)
          - self.set_qvalue (estado, acción, valor)
            que establece Q (estado, acción): = valor
        """

        self.get_legal_actions = get_legal_actions
        self._qvalues = defaultdict(lambda: defaultdict(lambda: 0))
        self.alpha = alpha
        self.epsilon = epsilon
        self.discount = discount

    def get_qvalue(self, state, action):
        """ Returns Q(state,action) """
        return self._qvalues[state][action]

    def set_qvalue(self,state,action,value):
        """ Sets the Qvalue for [state,action] to the given value """
        self._qvalues[state][action] = value

        
    def get_value(self, state):
        """
        Calcula la estimación de V (s) del agente utilizando los valores q actuales
        V (s) = max_over_action Q (estado, acción) sobre posibles acciones.
        Nota: tenger en cuenta que los valores q pueden ser negativos.
        """
        possible_actions = self.get_legal_actions(state)

        #SI NO HAY ACCIONES POSIBLES DEVOLVEMOS 0
        if len(possible_actions) == 0:
            return 0.0

        #QUEDATE CON EL VALOR DE LA ACCION QUE MAXIMICE Q-VALUE
        #TU CÓDIGO AQUI
        value = max(self.get_qvalue(state,action) for action in possible_actions)
        
        return value

    def update(self, state, action, reward, next_state):
        """
        ACTIALIZA EL Q-VALOR SEGUN LA FORMULA QUE SE PRESENTA:
           Q(s,a) := (1 - alpha) * Q(s,a) + alpha * (r + gamma * V(s'))
        """

        #PARAMETROS
        gamma = self.discount
        learning_rate = self.alpha

        #IMPLEMENTA LA FUNCION PRESENTADA EN LA DESCRIPCION DE ARRIBA
        value = (1-learning_rate) * self.get_qvalue(state,action) + learning_rate * (reward + gamma * self.get_value(next_state))
        
        self.set_qvalue(state, action, value)

    
    def get_best_action(self, state):
        """
        Calcula la mejor acción para tomar en un estado (utilizando los valores q actuales).
        """
        #SELECCIONA LAS POSIBLES ACCIONES DADO EL ESTADO
        possible_actions = self.get_legal_actions(state)

        #If there are no legal actions, return None
        if len(possible_actions) == 0:
            return None

        #CALCULA LA MEJOR ACCION DADO EL ESTADO
        #TU CODIGO AQUI
        best_action = None
        best_q = float("-inf")
        for action in possible_actions:
            cur_q = self.get_qvalue(state,action)
            if cur_q > best_q:
                best_q = cur_q
                best_action  = action

        return best_action

    def get_action(self, state):
        """
        Calcula la acción a tomar en el estado actual, incluida la exploración.
        Con probabilidad self.epsilon, deberíamos realizar una acción aleatoria.
        de lo contrario - la mejor acción política (self.getPolicy).
        """

        possible_actions = self.get_legal_actions(state)

        if len(possible_actions) == 0:
            return None

        #PARAMETROS
        epsilon = self.epsilon

        #SELECCIONA UNA ACCION TENIENDO EN CUENTA LA EXPLORACIÓN CON EL EPSILON
        #TU CODIGO AQUI
        if random.random() > epsilon:
            chosen_action = self.get_best_action(state)
        else:
            action = random.choice(possible_actions)
            chosen_action = action
        
        return chosen_action

In [ ]:


class EVSarsaAgent(QLearningAgent):
    
    def get_value(self, state):
        epsilon = self.epsilon
        possible_actions = self.get_legal_actions(state)


        if len(possible_actions) == 0:
            return 0.0
        
        #IMPLEMENTA EL VALOR MEDIO DE LAS ACCIONES.
        # TU CODIGO AQUI
        state_value = "XXXXXXXXXXXXXX"
        
        return state_value

# PROBEMOS NUESTRO MODELO SOBRE UN ENTORNO CONOCIDO.

In [ ]:
import gym, gym.envs.toy_text
env = gym.envs.toy_text.CliffWalkingEnv()
env = gym.make("Taxi-v3")
n_actions = env.action_space.n

print(env.__doc__)

In [ ]:
env.render()

In [ ]:

def play_and_train(env,agent,t_max=10**4):
    """This function should 
    - run a full game, actions given by agent.getAction(s)
    - train agent using agent.update(...) whenever possible
    - return total reward"""
    total_reward = 0.0
    s = env.reset()
    
    for t in range(t_max):
        a = agent.get_action(s)
        
        next_s,r,done,_ = env.step(a)
        agent.update(s, a, r, next_s)
        
        s = next_s
        total_reward +=r
        if done:break
        
    return total_reward

In [ ]:
from collections import defaultdict
import random, math
import numpy as np

agent_sarsa = EVSarsaAgent(alpha=0.25, epsilon=0.2, discount=0.99,
                       get_legal_actions = lambda s: range(n_actions))

agent_ql = QLearningAgent(alpha=0.25, epsilon=0.2, discount=0.99,
                       get_legal_actions = lambda s: range(n_actions))

In [ ]:
from IPython.display import clear_output
from pandas import DataFrame
moving_average = lambda x, span=100: DataFrame({'x':np.asarray(x)}).x.ewm(span=span).mean().values

rewards_sarsa, rewards_ql = [], []

for i in range(10000):
    rewards_sarsa.append(play_and_train(env, agent_sarsa))
    rewards_ql.append(play_and_train(env, agent_ql))
    
    if i %100 ==0:
        clear_output(True)
        print('EVSARSA mean reward =', np.mean(rewards_sarsa[-100:]))
        print('QLEARNING mean reward =', np.mean(rewards_ql[-100:]))
        plt.title("epsilon = %s" % agent_ql.epsilon)
        plt.plot(moving_average(rewards_sarsa), label='ev_sarsa')
        plt.plot(moving_average(rewards_ql), label='qlearning')
        plt.grid()
        plt.legend()
        plt.ylim(-500, 0)
        plt.show()
        

# VEAMOS LAS DIFERENCISA CON Q-LEARNING: Cliff Walking

In [ ]:
import gym, gym.envs.toy_text
env = gym.envs.toy_text.CliffWalkingEnv()
n_actions = env.action_space.n
#print(env.__doc__)

In [ ]:
env.render()

In [ ]:
agent_sarsa = EVSarsaAgent(alpha=0.25, epsilon=0.2, discount=0.99,
                       get_legal_actions = lambda s: range(n_actions))

agent_ql = QLearningAgent(alpha=0.25, epsilon=0.2, discount=0.99,
                       get_legal_actions = lambda s: range(n_actions))

In [ ]:
from IPython.display import clear_output
from pandas import DataFrame
moving_average = lambda x, span=100: DataFrame({'x':np.asarray(x)}).x.ewm(span=span).mean().values

rewards_sarsa, rewards_ql = [], []

for i in range(10000):
    rewards_sarsa.append(play_and_train(env, agent_sarsa))
    rewards_ql.append(play_and_train(env, agent_ql))
    #Note: agent.epsilon stays constant
    
    if i %100 ==0:
        clear_output(True)
        print('EVSARSA mean reward =', np.mean(rewards_sarsa[-100:]))
        print('QLEARNING mean reward =', np.mean(rewards_ql[-100:]))
        plt.title("epsilon = %s" % agent_ql.epsilon)
        plt.plot(moving_average(rewards_sarsa), label='ev_sarsa')
        plt.plot(moving_average(rewards_ql), label='qlearning')
        plt.grid()
        plt.legend()
        plt.ylim(-500, 0)
        plt.show()

In [ ]:
def draw_policy(env, agent):
    n_rows, n_cols = env._cliff.shape
    
    actions = '^>v<'
    
    for yi in range(n_rows):
        for xi in range(n_cols):
            if env._cliff[yi, xi]:
                print(" C ", end='')
            elif (yi * n_cols + xi) == env.start_state_index:
                print(" X ", end='')
            elif (yi * n_cols + xi) == n_rows * n_cols - 1:
                print(" T ", end='')
            else:
                print(" %s " % actions[agent.get_best_action(yi * n_cols + xi)], end='')
        print()

In [ ]:
print("Q-Learning")
draw_policy(env, agent_ql)

print("SARSA")
draw_policy(env, agent_sarsa)